In [2]:
import scraper
from importlib import reload

reload(scraper)
from scraper import *

In [3]:
driver = initialize_otodom_scraper()

In [4]:
first_page_url = 'https://www.otodom.pl/pl/wyniki/wynajem/mieszkanie/mazowieckie/warszawa/warszawa/warszawa?distanceRadius=0&limit=36&extras=%5BIS_PRIVATE_OWNER%5D&by=DEFAULT&direction=DESC&viewType=listing&page=2'
announcements_links = get_announcements_links(driver, first_page_url)

In [6]:
announcements_links[605]

'https://www.otodom.pl/pl/oferta/kawalerka-w-centrum-metro-arsenal-ID4pyCy'

In [ ]:
def click_button_by_text(driver, text):

In [9]:
text = 'Pokaż więcej'
button = driver.find_element(By.XPATH, f"//button[.//span[contains(text(), '{text}')]]")
button.click()

In [11]:
"//button[.//span[contains(text(), '{button_text}')]]".replace({"button_text" : button_text})

NameError: name 'button_text' is not defined

In [10]:
driver.find_element(By.XPATH, f'//div[@data-cy="adPageAdDescription"]').text

'Bardzo jasna kawalerka położona na starym Muranowie w cichej, zielonej enklawie w ścisłym centrum miasta.\nWynajem maksymalnie dla 1 osoby, niepalącej, bez zwierząt.\n  Minimum na rok.\nOpłaty miesięczne:\n2740zł to koszt wynajmu wraz z czynszem administracyjnym.\nDodatkowo płatny tylko prąd i gaz w wysokości łącznie około 70zł\n\nKaucja w wysokości 2740zł.\n\nMieszkanie składa się z przedpokoju połączonego z aneksem kuchennym, pokoju dziennego oraz łazienki. Znajduje się na trzecim piętrze niedawno odnowionej kamienicy. Nie ma windy.\nAneks kuchenny jest wyposażony w lodówkę z zamrażalnikiem, kuchnię gazową i szafki.\nPokój dzienny posiada bardzo duże dwuskrzydłowe okna do samej ziemi. Z okien widok na otwartą przestrzeń: korony drzew i dachy Śródmieścia. Pokój wyposażony jest w szafę na ubrania, komodę, biurko oraz rozkładaną kanapę z pojemnikiem na pościel.\nW łazience kabina prysznicowa, szafka pod umywalką i pralka. Ogrzewanie miejskie (nie ma piecyka).\nMiejsce jest doskonale sk